In [2]:
!pip install --upgrade google-cloud-storage

In [1]:
import os 
from google.cloud import storage

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='halocamp-477806-4de9060b540b.json'
storage_client = storage.Client()
dir(storage_client)

['SCOPE',
 '_SET_PROJECT',
 '__annotate_func__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_connection',
 '_batch_stack',
 '_bucket_arg_to_bucket',
 '_client_cert_source',
 '_connection',
 '_credentials',
 '_delete_resource',
 '_determine_default',
 '_extra_headers',
 '_get_resource',
 '_http',
 '_http_internal',
 '_initial_client_info',
 '_initial_client_options',
 '_is_emulator_set',
 '_list_resource',
 '_patch_resource',
 '_pop_batch',
 '_post_resource',
 '_push_batch',
 '_put_resource',
 '_universe_domain',
 'api_endpoint',
 'batch',
 'bucket',
 'close',
 'create_anonymous_client',
 'create_

### CREATE NEW BUCKET

In [3]:
bucket_name = 'fmcg-data-lake'
bucket = storage_client.bucket(bucket_name)
bucket.location = 'US'
bucket = storage_client.create_bucket(bucket)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14164\2615547250.py:3: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = 'US'


Conflict: 409 POST https://storage.googleapis.com/storage/v1/b?project=halocamp-477806&prettyPrint=false: Your previous request to create the named bucket succeeded and you already own it.

### INGESTION FROM GITHUB

In [4]:
def ingest_github_to_gcs():

    import requests
    from google.cloud import storage
    from datetime import datetime

    # ✅ RAW GitHub base (BUKAN /tree/)
    GITHUB_BASE_URL = (
        "https://raw.githubusercontent.com/"
        "Putriregitas/fmcg_project/main/raw"
    )

    FILES = [
        "raw_sales_transaction.csv",
        "raw_inventory_outlet_daily.csv",
        "raw_inventory_warehouse_daily.csv",
        "raw_outlet_master.csv",
        "raw_product_master.csv"
    ]

    BUCKET_NAME = "fmcg-data-lake"
    GCS_RAW_PATH = "raw"
    INGEST_DATE = datetime.today().strftime("%Y-%m-%d")

    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)

    for file_name in FILES:
        print(f"\nProcessing {file_name}...")

        url = f"{GITHUB_BASE_URL}/{file_name}"
        response = requests.get(url)

        # ❌ download failed
        if response.status_code != 200:
            print(f"❌ Failed to download {file_name}")
            continue

        # ❌ HTML instead of CSV
        if response.text.lstrip().startswith("<!DOCTYPE html"):
            raise ValueError(
                f"{file_name} is HTML, not CSV. "
                f"Check GitHub RAW URL."
            )

        dataset_name = file_name.replace("raw_", "").replace(".csv", "")
        gcs_path = (
            f"{GCS_RAW_PATH}/"
            f"{dataset_name}/"
            f"ingest_date={INGEST_DATE}/"
            f"{file_name}"
        )

        blob = bucket.blob(gcs_path)
        blob.upload_from_string(
            response.text,
            content_type="text/csv"
        )

        print(f"✅ Uploaded to gs://{BUCKET_NAME}/{gcs_path}")


In [5]:
ingest_github_to_gcs()


Processing raw_sales_transaction.csv...
✅ Uploaded to gs://fmcg-data-lake/raw/sales_transaction/ingest_date=2026-01-20/raw_sales_transaction.csv

Processing raw_inventory_outlet_daily.csv...
✅ Uploaded to gs://fmcg-data-lake/raw/inventory_outlet_daily/ingest_date=2026-01-20/raw_inventory_outlet_daily.csv

Processing raw_inventory_warehouse_daily.csv...
✅ Uploaded to gs://fmcg-data-lake/raw/inventory_warehouse_daily/ingest_date=2026-01-20/raw_inventory_warehouse_daily.csv

Processing raw_outlet_master.csv...
✅ Uploaded to gs://fmcg-data-lake/raw/outlet_master/ingest_date=2026-01-20/raw_outlet_master.csv

Processing raw_product_master.csv...
✅ Uploaded to gs://fmcg-data-lake/raw/product_master/ingest_date=2026-01-20/raw_product_master.csv
